In [1]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00


In [2]:
import rdflib
from rdflib import Graph

In [3]:
#### REQ1 #####
#### The KG must conceptualize the food domain at a fine-grained level.
path_to_recipes1m_usda = "https://horus-ai.fbk.eu/fuskg/ontology/abox/recipes/fuskg-usda-recipes-recipe1m.ttl"
recipes1m_usda_graph = Graph()
recipes1m_usda_graph.parse(path_to_recipes1m_usda, format="ttl")

<Graph identifier=Nd7f6f5f515104afd84b602081c798821 (<class 'rdflib.graph.Graph'>)>

In [4]:
## CQ1-1 ##
# Which are the ingredients of recipe X?
recipe_X = "Yogurt Parfaits"
q1_1 = recipes1m_usda_graph.query(
    """
    SELECT ?foodLabel
    WHERE {{
        ?recipe a fsreci:Recipe1m ;
                rdfs:label "{}" ;
                fsreci:hasRecipeFood ?recipefood .
        ?recipefood fsfood:hasFood ?food .
        ?food rdfs:label ?foodLabel .
    }}
""".format(recipe_X)
)
for result in q1_1:
    print("Ingredient: {}".format(result[0]))

Ingredient: Strawberries, raw
Ingredient: Yogurt, Greek, plain, nonfat (Includes foods for USDA's Food Distribution Program)
Ingredient: Cereals ready-to-eat, granola, homemade


In [5]:
## CQ1-2 ##
# Which are the nutrients of the food Y?
ingredient_Y = "Strawberries, raw"
q1_2 = recipes1m_usda_graph.query(
    """
    SELECT ?nutrClass
    WHERE {{
        ?food rdfs:label "{}"@en ;
              fscore:hasNutrient ?nutrInst .
        ?nutrInst a ?nutrClass .
        FILTER(STRSTARTS(str(?nutrClass), "https://w3id.org/fuskg/usda#usda-nut")) .
    }}
""".format(ingredient_Y)
)

for result in q1_2:
    print("Nutrient: {}".format(result[0]))

Nutrient: https://w3id.org/fuskg/usda#usda-nut-1003
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1004
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1005
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1007
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1009
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1010
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1011
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1012
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1013
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1014
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1018
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1057
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1058
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1075
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1079
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1087
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1089
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1090
Nutrient: https://w3id.org/fuskg/usda#usda-nut-1091
Nutrient: ht

In [6]:
## CQ1-3 ##
# How many calories does food Z have?
ingredient_Z = "Mushrooms, portabella, raw"
q1_3 = recipes1m_usda_graph.query(
    """
    SELECT ?food ?amountCalories
    WHERE {{
        ?food rdfs:label "{}"@en ;
              fsfood:amountCalories ?amountCalories .
    }}
""".format(ingredient_Z)
)

for result in q1_3:
    print("Amount of calories: {}".format(result[1]))

Amount of calories: 22.0


In [7]:
## CQ1-4 ##
# Which are the ingredients of recipe X that contain nutrient Y?

In [8]:
#### REQ2 ####
### The KG must provide a comprehensive list of activities that a person can perform. For each activity,
### it must be provided the knowledge required to understand the effort necessary to complete the activity
### to enable the inference of how much of each food defined under REQ1 is necessary to fulfill the activity.
path_to_activity_graph = "https://horus-ai.fbk.eu/fuskg/ontology/fuskg-activity.ttl"
activity_graph = Graph()
activity_graph.parse(path_to_activity_graph, format="ttl")

<Graph identifier=N9b66a81b75d64a658eafad2cd327a934 (<class 'rdflib.graph.Graph'>)>

In [9]:
## CQ2-1 ##
# Which is the effort needed to fulfill the activity X?
activity_X = "Salsa dancing"
q2_1 = activity_graph.query(
    """
    SELECT ?metValue
    WHERE {{
        ?activity a fsacti:Dancing ;
                 rdfs:label "{}"@en ;
                 fsacti:hasMETValue ?metValue .
    }}
    """.format(activity_X)
)

for result in q2_1:
    print("Activity {} has MET Value {}".format(activity_X, result[0]))

Activity Salsa dancing has MET Value 4.5


In [12]:
## CQ2-2 ##
# How many calories are consumed when performing activity Y for N minutes?
activity_Y = "mountain climbing"
N = 30
q2_2 = activity_graph.query(
    """
    SELECT ?KcalValue (?KcalValue * {} AS ?KcalValue30m)
    WHERE {{
        ?activity a fsacti:Sports ;
                 rdfs:label "{}"@en ;
                 fsacti:hasUnitKCalValue ?KcalValue .
    }}
    """.format(N, activity_Y)
)

for result in q2_2:
    print("Activity {} has {} Kcal per minute and when performed for {} minutes {} Kcal are consumed".format(activity_Y, round(float(result[0]), 2), N, round(float(result[1]), 2)))

Activity mountain climbing has 0.13 Kcal per minute and when performed for 30 minutes 4.0 Kcal are consumed


In [13]:
## CQ2-3 ##
# Which is the most intense activity of category C?
category_C = "WaterActivities"
q2_3 = activity_graph.query(
    """
    SELECT ?activityLabel ?maxMetValue
    WHERE {{
        {{
            SELECT (MAX(?metValue) as ?maxMetValue)
            WHERE {{
                ?activity a fsacti:{} ;
                         fsacti:hasMETValue ?metValue .
            }}
        }}
        ?activity a fsacti:{} ;
                 rdfs:label ?activityLabel ;
                 fsacti:hasMETValue ?maxMetValue .
        FILTER(LANG(?activityLabel) = 'en')
    }}
    """.format(category_C, category_C)
)
for result in q2_3:
    print("The most intense activity of category {} is {} with MET value {}".format(category_C, result[0], result[1]))

The most intense activity of category WaterActivities is free diving, vigorous effort with MET value 15.8
The most intense activity of category WaterActivities is scuba diving, vigorous effort with MET value 15.8


In [14]:
## CQ2-4 ##
# Which are the activities of category C that consume at least 0.1 kcal?
q2_4 = activity_graph.query(
    """
    SELECT ?activityLabel ?KcalValue
    WHERE {{
        ?activity a fsacti:{} ;
                 rdfs:label ?activityLabel ;
                 fsacti:hasUnitKCalValue ?KcalValue .
        FILTER(LANG(?activityLabel) = 'en' && ?KcalValue >= 0.1)
    }}
    ORDER BY ASC(?activityLabel)
    """.format(category_C)
)
for result in q2_4:
    print("Activity {} consumes {}".format(result[0], round(float(result[1]),2)))

Activity backstroke swimming competition consumes 0.16
Activity backstroke swimming training consumes 0.16
Activity backstroke swimming, moderate effort consumes 0.16
Activity breaststroke swimming competition consumes 0.16
Activity breaststroke swimming training consumes 0.16
Activity breaststroke swimming, moderate effort consumes 0.17
Activity butterfly swimming consumes 0.23
Activity canoeing crew in competition consumes 0.2
Activity crawl swimming, vigorous effort consumes 0.17
Activity free diving, light effort consumes 0.12
Activity free diving, moderate effort consumes 0.2
Activity free diving, vigorous effort consumes 0.26
Activity freestyle swimming, moderato effort consumes 0.14
Activity freestyle swimming, vigorous effort consumes 0.16
Activity kayaking competition consumes 0.21
Activity kayaking, vigorous effort consumes 0.21
Activity kitesurfing, moderate effort consumes 0.18
Activity kitesurfing, vigorous effort consumes 0.23
Activity open water swimming (lake, sea, rive

In [ ]:
#### REQ3 #####
### The KG must include the model of the barriers that may affect a person and how such barriers obstacle
### the fulfillment of specific activities, the consumption of specific foods, or, in general, the following of
### specific guidelines.

In [ ]:
## CQ3-1 ##
# Why user U cannot perform activity X?

In [ ]:
## CQ3-2 ##
# Why user U cannot consume food Y?

In [ ]:
## CQ3-3 ##
# Which are the barriers preventing the fulfillment of activity Z?

In [15]:
#### REQ4 #####
### The KG must support the modeling of multi-modal knowledge since a MMKG may be exploited to better support
### users’ education tasks and enable knowledge injection tasks into large foundational models.
path_to_recipes1m_multimodal_graph = "https://horus-ai.fbk.eu/fuskg/ontology/abox/multimodal/fuskg-multimodal-recipe1m.ttl"
multimodal_graph_recipe1m = Graph()
multimodal_graph_recipe1m.parse(path_to_recipes1m_multimodal_graph, format="ttl")

<Graph identifier=N92ba0d70ca4c4829807bfe0ce18edc2c (<class 'rdflib.graph.Graph'>)>

In [16]:
## CQ4-1 ##
# Where is it possible to find a picture of recipe X?
recipe_X = "Yogurt Parfaits"
q4_1 = multimodal_graph_recipe1m.query(
    """
    SELECT ?recipe ?modalDescriptor ?resourceURL
    WHERE {{
        ?recipe a fsreci:Recipe1m ;
                rdfs:label "{}" .
        ?recipe fsmumo:hasModalDescriptor ?modalDescriptor .
        ?modalDescriptor fsmumo:resourceURL ?resourceURL .
    }}
""".format(recipe_X)
)

for result in q4_1:
    print("ResourceURL: {}".format(result[2]))

ResourceURL: https://i.ytimg.com/vi/G1wcU5aBOOk/maxresdefault.jpg
ResourceURL: https://kidseatsmartnl.files.wordpress.com/2014/10/yogurt-parfait-oct2014.jpg
ResourceURL: http://del.h-cdn.co/assets/cm/15/10/54f9287d78ad7_-_yogurtparfait.jpg
ResourceURL: https://s-media-cache-ak0.pinimg.com/736x/49/8d/c3/498dc3ed97cf3a8c588d29492d76671b.jpg
ResourceURL: http://tastykitchen.com/recipes/wp-content/uploads/sites/2/2011/06/IMG_2692-630x420-410x615.jpg
ResourceURL: http://www.tastearkansas.com/wp-content/uploads/2013/07/RaspberyParfait2.jpg
ResourceURL: https://static01.nyt.com/images/2013/06/25/science/28recipehealth/28recipehealth-articleLarge.jpg
ResourceURL: https://images-gmi-pmc.edge-generalmills.com/68970b26-cc64-4fc1-b6c5-457362c699e7.jpg
ResourceURL: http://lmld.org/wp-content/uploads/2013/05/054editedsmall.jpg


In [ ]:
## CQ4-2 ##
# Where is it possible to download a video showing how to perform activity Y? --- ongoing finding data

In [ ]:
#### REQ5 #####
### Knowledge modeled under the requirements REQ1, REQ2, REQ3, and REQ4 must be associated with knowledge and
### data gathered from users’ input.

In [ ]:
## CQ5-1 ##
# What did the user U consume N days ago for breakfast?

In [ ]:
## CQ5-2 ##
# How many times did user U perform activity Y last week?

In [ ]:
## CQ5-3 ##
# Did user U consume food Y today?

In [ ]:
## CQ5-4 ##
# How many calories were consumed by user U yesterday?

In [ ]:
#### REQ6 #####
### A KG usable for creating a behavior change solution requires a set of guidelines driving the behavior change intervention.

In [ ]:
## CQ6-1 ##
# How many times is it possible to consume food X in a week?

In [ ]:
## CQ6-2 ##
# How many minutes of activity Y should be performed by user U in a single training session?

In [ ]:
## CQ6-3 ##
# How much food Y can be eaten during a single meal?

In [ ]:
## CQ6-4 ##
# Is it allowed to consume M grams of food Z in a single day?

In [19]:
#### REQ7 #####
### The KG must include temporal knowledge to support the requirements above and enable temporal reasoning over the users’ collected knowledge.
path_to_tasty_rsteps_graph = "https://horus-ai.fbk.eu/fuskg/ontology/abox/recipes-steps/fuskg-recipes-recipessteps-tasty.ttl"
tasty_rsteps_graph = Graph()
tasty_rsteps_graph.parse(path_to_tasty_rsteps_graph, format="ttl")

<Graph identifier=N414b0a9abcd144fb80def7a15874ebab (<class 'rdflib.graph.Graph'>)>

In [20]:
## CQ7-1 ##
# Which is the step following step X during the preparation of recipe Y?
recipe_Y = "sofritas-burrito-bowl"
step_X = "add the soy sauce, chili powder, garlic powder, cumin, paprika, and cayenne."
q7_1 = tasty_rsteps_graph.query(
    """
    SELECT ?recipeStepAfterLabel
    WHERE {{
        ?recipe a fsreci:Tasty ;
                 rdfs:label "{}" ;
                 fstemp:hasFirstRecipeStep ?firstStep .
        ?firstStep fstemp:afterStep* ?recipeStepBefore .
        ?recipeStepBefore fstemp:hasStepDescription ?stepBeforeDescriptionStep .
        ?stepBeforeDescriptionStep fstemp:stepDescription "{}"^^xsd:string.
        ?recipeStepBefore fstemp:afterStep ?recipeStepAfter.
        ?recipeStepAfter fstemp:hasStepDescription ?StepAfterDescriptionStep .
        ?StepAfterDescriptionStep fstemp:stepDescription ?recipeStepAfterLabel .
    }}
    """.format(recipe_Y, step_X)
)

for result in q7_1:
    print("Step following '{}' is:\n\n {}".format(step_X, result[0]))

Step following 'add the soy sauce, chili powder, garlic powder, cumin, paprika, and cayenne.' is:

 continue to cook and break the tofu apart until browned and crumbly.


In [ ]:
## CQ7-2 ##
# Which is the second movement to do when performing activity Z? --- ongoing finding data

In [ ]:
## CQ7-3 ##
# Which are the timespans used to build guidelines?